In [360]:
import import_ipynb
import pexpect
import sys
import numpy as np
%run PrintFunctions.ipynb
%run Names.ipynb
%run NeuralNetInputFunctions.ipynb
%run LogitFiltering.ipynb
np.set_printoptions(threshold=sys.maxsize)

In [357]:
STARTCMD = "/Users/weston/pokemon-showdown-master/pokemon-showdown simulate-battle"
STARTGAMECMD = '>start {"formatid":"gen8ou"}'
GENTEAMCMD = '/Users/weston/pokemon-showdown-master/pokemon-showdown generate-team {"formatid":"gen8ou"}'

In [358]:
class ProcessFunctions:
    def startProcess():
        '''
        Starts a tty for the game
        '''
        formattedPrint("Starting process", location = "PROCESSFUNCTIONS")
        process = pexpect.spawn(STARTCMD)
        process.delaybeforesend = None
        return process
    def startGame(process):
        '''
        Sends a command to the tty to start the game
        '''
        ProcessFunctions.sendString(process, STARTGAMECMD)
    def clear(process):
        '''
        Sends a unique string to the processs, expects it, and clears output
        '''
        uniqueStr = "Heart been broke so many times I i dont know what to believvveeee"
        process.send(uniqueStr)
        process.expect(uniqueStr)
    def sendString(process, characters, expect=False):
        '''
        Sends a string and expects it later, to clear the characters from the buffer
        '''
        process.sendline(characters)
        if expect:
            process.expect(characters)
    def getOutput(process, end = '\r\n\r\n', numTimes = 1):
        '''
        Gets output from the tty, output is NOT processed
        '''
        output = ""
        for i in range(numTimes):
            process.expect(end, timeout = 1)
            output += process.before.decode()
        return output
    def processOutput(output):
        '''
        Processes output into a list of lines
        '''
        output = output.strip()
        lines = output.split("\n")
        return lines
    def getProcessedOutput(process, end = '\r\n\r\n', numTimes = 1):
        '''
        Gets output for a process, end is the delimiter to get the input before
        '''
        output = ProcessFunctions.getOutput(process, end = end, numTimes = numTimes)
        lines = ProcessFunctions.processOutput(output)
        for index in range(len(lines)):
            lines[index] = lines[index].rstrip()
        return lines
    def getRandomTeam():
        '''
        Uses a separate process to generate a team and returns the output
        '''
        process = pexpect.spawn(GENTEAMCMD)
        while True:
            try:
                out = ProcessFunctions.getProcessedOutput(process, end = '\r\n')
                break
            except pexpect.exceptions.TIMEOUT:
                pass
        process.terminate()
        return out[0]
    def setPlayer(process, team, name, num):
        '''
        Sets a player for the current process with a team and name
        '''
        if num != 1 and num != 2:
            raise ValueError('Player number must be 1 or 2')
        cmd = '>player p' + str(num) + ' {"name":"' + name + '","team":' + '"' + team + '"''}'
        ProcessFunctions.sendString(process, cmd)
    def doMove(process, playerNum, moveNum, canonicalNames, currentNames):
        '''
        Maps the output of the neural net to the cmd for the move and sends it to the tty
        0: move 1
        1: move 2
        2: move 3
        3: move 4
        4: switch 1
        5: switch 2
        6: switch 3
        7: switch 4
        8: switch 5
        9: switch 6
        '''
        switch = False
        moveNum += 1
        if moveNum > 4:
            moveNum = moveNum - 4
            nameDict = ValidationFunctions.getNameDict(canonicalNames, currentNames)
            moveNum = nameDict[moveNum]
            switch = True
        cmd = " switch " if switch else " move "
        print('>p' + str(playerNum) + cmd + str(moveNum))
        process.sendline('>p' + str(playerNum) + cmd + str(moveNum))
    def getTurnOutput(process):
        '''
        Cleaner abstraction of getProcessedOutput, dont have to type \|upkeep everytime
        '''
        try:
            out = ProcessFunctions.getProcessedOutput(process, end = ["\|turn", "\|upkeep"])
            if out[0] == '':
                out = ProcessFunctions.getProcessedOutput(process, end = ["\|turn", "\|upkeep"])
        except pexpect.exceptions.TIMEOUT:
            out = []
            while True:
                try:
                    curOut = ProcessFunctions.getProcessedOutput(process, end = ["\r\n\r\n"])
                except pexpect.exceptions.TIMEOUT:
                    return out
                out.extend(curOut)
        return out
    def setTeams(process, name1 = '', name2 = '', team1 = None, team2 = None):
        if team1 is None:
            team1 = ProcessFunctions.getRandomTeam()
        if team2 is None:
            team2 = ProcessFunctions.getRandomTeam()
        if name1 == '':
            name1 = Names.getRandomName()
        ProcessFunctions.setPlayer(process, team1, name1, 1)
        print(ProcessFunctions.getProcessedOutput(process))
        if name2 == '':
            name2 = Names.getRandomName()
        ProcessFunctions.setPlayer(process, team2, name2, 2)
        return name1, name2
    def testRandomPlayerMove(p, pDict, playernum, canonicalNames):
        if 'wait' not in pDict: #p1 move
            moveNum = ValidationFunctions.testMove(pDict, canonicalNames)
            print("mNum")
            print(moveNum)
            pcurrentNames = list(pDict['names'].keys())
            ProcessFunctions.doMove(p, playernum, int(moveNum), canonicalNames, pcurrentNames)
    def testLinesForWin(outputLines, name1, name2):
        for line in outputLines:
            if "|win|" in line:
                if name1 in line:
                    return 1
                else:
                    return 2
        return 0
    def startGameAndGetDicts(name1 = '', name2 = '', team1 = None, team2 =  None):
        p = ProcessFunctions.startProcess() # start process
        ProcessFunctions.startGame(p) # start game
        print(ProcessFunctions.getProcessedOutput(p, end = "\n\r"))
        name1, name2 = ProcessFunctions.setTeams(p, name1 = name1, name2 = name2, team1 = team1, team2 = team2) # set teams
        output1 = ProcessFunctions.getProcessedOutput(p, end = "\r\n\r\n")
        output2 = ProcessFunctions.getProcessedOutput(p, end = "\r\n\r\n")
        gameEffectDict, p1Dict, p2Dict = ScrapingFunctions.getDicts(output1 + output2, {'conditions' : ['',''],
                                                                                       'p1sideeffect' : [],
                                                                                       'p2sideeffect' : []})
        return p, gameEffectDict, p1Dict, p2Dict, name1, name2